In [1]:
##Required Packages 
import os
import sys
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import DutchStemmer
import matplotlib.pyplot as plt
import spacy
import tiktoken
import json
import numpy as np
from scipy.special import softmax
import markdown
from langdetect import detect
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
####

nltk.download('stopwords')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\samge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
#CHANGE FILENAME
data = pd.read_csv("Reviews.csv")


C:\Users\samge\AppData\Local\Temp\ipykernel_5552\1363499268.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Reviews.csv")


In [5]:
training_data = data.dropna(subset = [ "NPS driver", "NPS driver explanation"])
def word_count(text):
    return len(str(text).split())

# Create a new column for word counts (optional, for inspection)
training_data['NPS_word_count'] = training_data['NPS driver explanation'].apply(word_count)

# Filter rows where word count is 5 or more
training_data = training_data[training_data['NPS_word_count'] >= 5]

C:\Users\samge\AppData\Local\Temp\ipykernel_5552\3318353539.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['NPS_word_count'] = training_data['NPS driver explanation'].apply(word_count)


In [6]:
def detect_language(text):
    try: 
        return detect(text)
    except:
        return "unknown"
    
training_data["Language"] = training_data["Review general"].apply(detect_language)


training_english_reviews = training_data[training_data["Language"] == "en"]
training_dutch_reviews = training_data[(training_data["Language"] == "nl")| (training_data["Language"] == "af")]

In [7]:
#####################################
#### English LANGUAGE PREPROCESSING ##
######################################

# Load the English spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize English stopwords
stop_words = set(stopwords.words('english'))

def preprocess_English(text):
    """
    Cleans, normalizes, and applies lemmatization to English text.
    
    :param text: A string of text to preprocess.
    :return: Preprocessed text as a string.
    """
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove URLs and any non-textual content
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    
    # Normalize text
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    
    # Tokenize and remove stop words
    tokens = word_tokenize(text, language='english')
    tokens = [word for word in tokens if word not in stop_words]
    
    # Apply lemmatization using spaCy
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]
    
    # Rejoin tokens into a single string
    return ' '.join(lemmas)

In [8]:
training_english_reviews["P_Explanation"] = training_english_reviews["NPS driver explanation"].apply(preprocess_English)


C:\Users\samge\AppData\Local\Temp\ipykernel_5552\3384719694.py:19: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
C:\Users\samge\AppData\Local\Temp\ipykernel_5552\3076933442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_english_reviews["P_Explanation"] = training_english_reviews["NPS driver explanation"].apply(preprocess_English)


In [16]:
#CREATING DICTIONARY WITH COUNTS
nps_driver_texts = training_english_reviews.groupby("NPS driver")["P_Explanation"].apply(lambda x: " ".join(x)).to_dict()
drivers = list(nps_driver_texts.keys())
reviews = list(nps_driver_texts.values())
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(reviews)
dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out(), index=drivers)
likelihood_df = pd.DataFrame(columns=drivers)

for word in vectorizer.get_feature_names_out():
    driver_counts = dtm_df[word].tolist()  # Counts for each topic
    smoothed = [x + 1 for x in driver_counts]  # Apply Laplace smoothing
    likelihood_df.loc[word] = smoothed

# Step 5: Normalize the word counts to calculate likelihoods
for driver in drivers:
    total_word_count = likelihood_df[driver].sum()
    likelihood_df[driver] = likelihood_df[driver] / total_word_count

# Step 6: Add words as a column and reset the index
likelihood_df["words"] = likelihood_df.index
likelihood_df = likelihood_df.reset_index(drop=True)

# Step 7: Extract the lexicon content
lexicon = likelihood_df["words"].astype(str).tolist()
likelihood_df.to_csv("dict.csv")

lexicon = pd.DataFrame.from_dict(lexicon)
lexicon.to_csv("Lexicon.csv")

In [11]:
driver_occurence = training_english_reviews["NPS driver"].value_counts()
prior_probabilities = driver_occurence/(len(training_english_reviews))
prior_probabilities.to_csv("probs.csv")